In [1]:
from openai import OpenAI
import os
import json
import time

In [2]:
client = OpenAI(api_key = os.environ['OPENAI_API_KEY'])

In [3]:
f = open('system_prompt.txt','r')
system_prompt = f.read()
f.close()
f = open('world_view', 'r')
world_view = f.read()
f.close()

In [12]:
# # create new assistant object
# assistant = client.beta.assistants.create(
#     name="Story Generator",
#     instructions = system_prompt,
#     response_format={"type": "json_object"},
#     model="gpt-4o",
# )
# ASSISTANT_ID = assistant.id

# Using pre-made assistant
STORY_ID = 'asst_QJULNvgbgB8cU1YgY5JzLAyr'
SKILL_ID = 'asst_JSej8B49OeeI7IwOUNp3dpKY'

In [5]:
player_status = '''{
  'Player Name': 'Argon', 
  'status': {'HP':200, 'MP':100, 
             'Strength':30, 'Skill':15, 
             'Dexterity': 5, 'Intelligence': 5, 
             'Luck': 5, 'Defense': 30, 
             'Speed': 5, 'Concentration': 20, 
             'Reaction': 15, 'MP Regeneration': 2, 'HP Regeneration': 2},
  'Gold': 1000, 
  'Level': 10, 
  'Gender': 'Male', 
  'Social Status': 'Commoner', 
  'Strength Level': '2-star Knight', 
  'Equipment': { 'Top': {'Item Name': 'Ordinary Leather Armor', 'Use Restriction': None, 'Effect': {'Defense':10, 'Dexterity': -3}}, 
                'Weapon': {'Item Name': 'Iron Sword', 'Use Restriction': {'Strength':15}, 'Effect': {'Strength':10}} } 
}'''

In [6]:
start_message = [
  {
    "type": "text",
    "text": world_view
  },
  {
    "type": "text",
    "text": player_status
  }
]

In [15]:
def wrap_text(text, width):
    for i in range(0, len(text), width):
        print(text[i:i + width])
        
def show_message(json_object):
  if 'Story' in json_object:
    wrap_text(json_object['Story'], 40)
    for idx, choice in enumerate(json_object['Choices']):
      print(f'{idx+1}: {choice["text"]} ({choice["status"]}), Gold:({choice["gold"]}), ({choice["next_type"]})')
  else:
    display(json_object)
    
def run_thread(user_message, thread, assistant_id):
  run = submit_message(user_message, thread, assistant_id)
  run = wait_run(run, thread)
  messages = get_message(thread)
  text = messages.data[0].content[0].text.value
  json_object=json.loads(text)
  show_message(json_object)
  return json_object


def start_thread(thread, start_message, assistant_id):
  # create thread and add first message in thread
  run_thread(start_message, thread, assistant_id)

  for i in range(5):
    user_message = input()
    print(user_message)
    if user_message == "quit":
      break
    response = run_thread(user_message, thread, assistant_id)
    if 'Combat' in response:
      run_thread(str(response['Combat']), thread, SKILL_ID)


def submit_message(user_message, thread, assistant_id):
  client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message
  )
  run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
  )
  return run


def wait_run(run, thread):
  while run.status == "queued" or run.status == "in_progress":
    # connect thread and assistant
    run = client.beta.threads.runs.retrieve(
      thread_id=thread.id,
      run_id=run.id
    )
    time.sleep(0.5)
  return run

def get_message(thread):
  return client.beta.threads.messages.list(thread_id=thread.id)

In [18]:
thread = client.beta.threads.create()

In [19]:
start_thread(thread,start_message,STORY_ID)

Argon found himself at the edge of the V
erdant Woods, a dense forest filled with
 ancient trees and hidden secrets. The d
ay was bright, yet a sense of unease lin
gered in the air, as though the forest i
tself was watching. Argon had heard rumo
rs of a hidden shrine deep within these 
woods, said to grant tremendous powers t
o those brave enough to find it. As he v
entured deeper into the forest, the sunl
ight grew dim, and the sounds of wildlif
e faded, replaced by an eerie silence.


Suddenly, Argon encountered an old hermi
t, who emerged from behind a gnarled oak
. The hermit spoke in a raspy voice, 'I 
see you're seeking the shrine. Be warned
, only those with skill and courage can 
survive its trials.' The hermit then off
ered Argon two options: a map to the shr
ine for a sum of gold or direct guidance
 to the shrine in exchange for a favor l
ater on. As Argon pondered his choices, 
the air around him grew colder, and the 
shadows seemed to move with a life of th
eir own.
1: Buy 

{'Combat': {'Monster Name': 'Forest Goblin',
  'Monster Description': 'A sneaky goblin with glowing eyes, adept at ambushing unsuspecting travelers in the Verdant Woods.',
  'status': {'HP': 150,
   'MP': 50,
   'Strength': 20,
   'Skill': 10,
   'Dexterity': 15,
   'Intelligence': 7,
   'Luck': 5,
   'Defense': 10,
   'Speed': 20,
   'Concentration': 10,
   'Reaction': 20,
   'MP Regeneration': 1,
   'HP Regeneration': 2}},
 'Reward': {'gold': 50, 'exp': 30},
 'Item': {'Item types': 'Weapon',
  'Item Name': 'Goblin Dagger',
  'Item Description': 'A small, sharp dagger looted from a forest goblin. It glows faintly with a sinister aura.',
  'Use Restriction': {'Dexterity': 10},
  'Effect': {'Strength': 5, 'Dexterity': 5}}}

{'Attack_Type': 'Physical',
 'Physical_Attack_Method': 'Stab',
 'Penetration': {'Physical_Defense_Penetration': '15%',
  'Magical_Defense_Penetration': '0%'},
 'Accuracy': '80%',
 'Status_Effect': {'Poison': {'Status': ['Skill'],
   'Coefficient': ['0.05'],
   'Duration': '3 turns'}},
 'Damage': {'Status': ['Strength'], 'Coefficient': ['1.2'], 'Count': '1'}}

victory
With a swift and powerful strike, Argon 
defeated the forest goblin. It collapsed
 onto the ground, its glowing eyes dimmi
ng as life left its body. Panting slight
ly, Argon sheathed his iron sword and ex
amined the goblin's remains. Among the c
reature's belongings, he found a small, 
sharp dagger that emitted a faint sinist
er glow. It seemed valuable, so Argon de
cided to take it.

With the forest gobli
n defeated, Argon continued his journey,
 feeling a renewed sense of determinatio
n. The path to the ancient oak was now c
lear, and he knew he had to stay alert f
or any more hidden threats. As he approa
ched the towering tree with twisted bran
ches, he noticed an inscription carved i
nto the bark. It detailed a riddle that 
needed to be solved to proceed: 'I speak
 without a mouth and hear without ears. 
I have no body, but I come alive with wi
nd. What am I?'

Argon pondered the ridd
le, deciding his next course of action s
hould he solve it or choose another path
 forward